In [1]:
"""
Tensorflowによるオートエンコーダの実装
思い出しがてらの作成なのでコメントが膨大…
"""


#%%
# cording = UTF-8
import os
import re
import random
import copy
import scipy
import librosa #無くしたい
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import keras
import tensorflow

####################################初期化####################################
aug_amount = 100    #ファイルごとのAugmentationの回数
lr = 1e-01          #初期学習率
alpha = 1e-03       #L2正則化の係数
dr_rate = 0.3       #ドロップアウト率
batch_size = 50
epochs = 100
encode_dim = 1000   #オートエンコーダの圧縮次元

#ディレクトリの初期化
base_dir = "../"
data_dir =os.path.join(base_dir,"data")
ok_dir = os.path.join(data_dir,"OK")
ng_dir = os.path.join(data_dir,"NG")
env_dir = os.path.join(data_dir,"environment")

#学習用データファイル
datafile = "dataset2.npz"

####################################関数定義###################################

#対象ディレクトリのファイル一覧を取得
def get_file_list(dir):
    path = dir
    file_list = os.listdir(path)
    print("get file_list :{}".format(file_list))
    return file_list

#対象ディレクトリの最大ファイルをサーチ
def wav_search(dir,f_list):
    #呼び出されるごとに初期化する
    wave_list = []
    file_size = 0
    
    return_path = os.path.abspath('./')
    
    os.chdir(dir)
    for i in f_list:
        search_index = re.search('.wav',i)
        if search_index:
            wave_list .append(i)
            if os.path.getsize(i) > file_size:
                file_size = os.path.getsize(i)
                largest_file = i
        
    os.chdir(return_path)   #カレントディレクトリを戻す
    print("get file :{0} ,file size:{1}"\
        .format(largest_file,file_size))
    return wave_list,largest_file,file_size

#オーディオファイルの読み込み サンプルレート22.05kHz、モノラルで固定
def load_wav(dir,file):
    #呼び出されるごとに初期化する
    wf = np.arange(0)

    f_path = os.path.join(dir,file)
    wf,sp_rate = librosa.load(f_path,sr=22050,mono = True)
    del sp_rate
    return wf

#スペクトログラムの取得 パワースペクトラムのまま処理するならlibrosa不要
def get_spg(wf):
    spg = np.arange(0)
    sp_f,sp_t,spg = scipy.signal.spectrogram(wf,fs=22050,
        window = np.hamming(1024),nfft =1024)
    spg = librosa.power_to_db(spg)
    spg =spg.astype('float16')
    return sp_f,sp_t,spg

#Augmentationの処理
def aug_process(frame,dir,wave_list,env_file,):
    #呼び出されるごとに初期化する
    length = 0
    count = 0
    wf = np.arange(0)

    length = int(frame * 1.2)
    for i in wave_list:
        wf = load_wav(dir,i)
        for j in range(aug_amount):
            start = random.randint(0,len(env_file)-length)
            aug_wav = copy.deepcopy(env_file[start : start + length])
            del start
            start = random.randint(0,len(aug_wav) - len(wf))
            aug_wav = aug_wav + random.gauss(1,0.05)
            aug_wav[ start:start + len(wf) ] = \
                aug_wav[ start : start + len(wf) ] + wf
            sp_f,sp_t,spg = get_spg(aug_wav)
            spg = spg.reshape(1,len(sp_f),len(sp_t))
            try:
                X_data
            except:
                X_data = copy.deepcopy(spg)
            else:
                X_data = np.vstack((X_data,spg))
            del start,aug_wav,sp_f,sp_t,spg
            count = count + 1
        del wf
        print("Augmentation done! total count = {}".format(count))

    return X_data

#データセットの作成 ここまでの関数は全部ここに集約される
#最大ファイルサイズに合わせてフレームサイズを定義し
#OK・NG各データセットを作成後、結合する

def new_dataset(aug,ok_dir,ng_dir,env_dir):
    #OKNGそれぞれのファイルリストと最大ファイルを取得
    ok_filelist = get_file_list(ok_dir)
    ok_wave_list,ok_largeest_name,ok_largest_size = wav_search(ok_dir,ok_filelist)
    ng_filelist = get_file_list(ng_dir)
    ng_wave_list,ng_largeest_name,ng_largest_size = wav_search(ng_dir,ng_filelist)

    #OKNGの最大を比較
    if ok_largest_size>ng_largest_size:
        largest_dir = ok_dir
        lergest_name = ok_largeest_name
        print("largetst:OK")
    else:
        largest_dir = ng_dir
        lergest_name = ng_largeest_name
        print("largetst:NG")

    #最大フレームサイズを取得
    wf = load_wav(largest_dir,lergest_name)
    frame = int(len(wf))
    #wf = np.insert(wf,frame,np.empty(int(frame*0.2))) #1.2倍する
    #sp_f,sp_t,spg = get_spg(wf) 
    #X_initsize = (len(sp_f),len(sp_t))
    #del wf,sp_f,sp_t,spg
    del wf

    #環境音データをロード
    env_data = load_wav(env_dir,"env.wav")
    
    #OKデータセット作成
    X_ok = copy.deepcopy(
        aug_process(frame,ok_dir,ok_wave_list,env_data)
        )
    y_ok = np.zeros(len(X_ok),dtype = 'bool')

    #NGデータセット作成
    X_ng = copy.deepcopy(
        aug_process(frame,ng_dir,ng_wave_list,env_data)
        )
    y_ng = np.ones(len(X_ng),dtype = 'bool')

    #データセットの結合
    X_data = np.vstack((X_ok,X_ng))
    y_data = np.append(y_ok,y_ng)
    del X_ok,y_ok,X_ng,y_ng

    return X_data,y_data

#OKNGが混在したデータからFalseのみを分離する
def mixed_to_sprit(X_mixed,y_mixed):
    #呼び出されるごとに初期化する
    try:
        X_sprit
    except:
        pass    #X_spritが存在しなければ何もしない
    else:
        del X_sprit #前のデータを消去する

    for i in range(len(X_mixed)):
        if y_mixed[i] == False:
            try:
                X_sprit
            except: #X_spritを生成する
                X_sprit = copy.deepcopy(X_mixed[i])
                X_sprit = X_sprit.reshape(1,X_mixed.shape[1])
            else:   #既存のX_spritに追加する 前段の例外処理はここを避けるため
                X_sprit = np.vstack(
                    (X_sprit,X_mixed[i].reshape(1,X_mixed.shape[1]))
                    )

    return X_sprit

###################################メイン処理###################################

#データセット読み込み なければ作る
if os.path.exists(os.path.join(data_dir,datafile)) == False:
    X_data,y_data = new_dataset(aug_amount,ok_dir,ng_dir,env_dir)
    np.savez_compressed(os.path.join(data_dir,datafile),
        X = X_data,y = y_data)
    print("Data set saved!") #ファイルネーム表示機能つけること
else:
    load_data = np.load(os.path.join(data_dir,datafile))
    X_data =load_data['X']
    y_data = load_data['y']
    del load_data
    print("Data loaded!!")

#データ前処理 trainとtestを分離
from sklearn.model_selection import train_test_split
X_shape = X_data.shape[1:]
X_data = X_data.reshape(len(X_data),-1) #アフィン変換
X_train,X_test,y_train,y_test = \
    train_test_split(X_data,y_data,test_size=0.05)
print(
"Training data amounts :{0}\n\
Test data amounts :{1}"\
.format(len(y_train),len(y_test))
)
del X_data,y_data


Using TensorFlow backend.
Data loaded!!
Training data amounts :3800
Test data amounts :200


In [2]:
#X_trainからOKデータ(False)だけを抽出する

X_train_ae = mixed_to_sprit(X_train,y_train)
print(
"X_train for Autoencoder was splited!!\n\
amount/shape:{0}"
.format(X_train_ae.shape)
)

#スケーラを定義する(AEの出力にシグモイドを使うため)
scaler = sklearn.preprocessing.MinMaxScaler()


X_train for Autoencoder was splited!!
amount/shape:(1901, 60021)


In [3]:
#オートエンコーダの定義

from keras.models import Model,Sequential
from keras.layers import Input, Dense,Dropout,Activation
from keras.layers.normalization import BatchNormalization
from keras import regularizers

#Modelでの実装 もう少しわかりやすい形にする
def ae(input_dim,encode_dim,lr =lr,alpha =alpha,dr_rate = dr_rate):

    input_data = Input(shape = (input_dim,))

    #エンコーダを定義
    encoder = Dense(
        encode_dim,
        kernel_initializer="he_normal",
        kernel_regularizer=regularizers.l2(alpha),
        )(input_data)
    encoder = BatchNormalization()(encoder)
    encoder = Dropout(dr_rate)(encoder)
    encoder = Activation("relu")(encoder)

    #デコーダを定義 こっちにはドロップアウトは定義しない
    decoder = Dense(input_dim,kernel_initializer="he_normal")(encoder)
    decoder = BatchNormalization()(decoder)
    decoder = Activation("sigmoid")(decoder)

    #モデルを定義
    autoencoder = Model(input = input_data,output = decoder)

    #最適化関数 Nadamをデフォルトパラメタで使う 必要ならlr = lr を追記
    opt = keras.optimizers.nadam()

    autoencoder.compile(
        optimizer = opt,loss='binary_crossentropy',metrics=['accuracy']
        )

    return autoencoder

#オートエンコーダの学習
ae = ae(
    input_dim = X_train_ae.shape[1],
    encode_dim = encode_dim
)                                       #インスタンス化

X_train_ae = scaler.fit_transform(X_train_ae)     #スケール変換

ae.fit(
    X_train_ae,X_train_ae,
    epochs = epochs,
    batch_size = batch_size,
    shuffle = True
)

print('Autoencoder learning is over!')
ae.summary()


Epoch 1/100
1901/1901 [==============================] - 114s 60ms/step - loss: 0.8847 - accuracy: 0.0200
Epoch 2/100
1901/1901 [==============================] - 114s 60ms/step - loss: 0.7713 - accuracy: 0.0216
Epoch 3/100
1901/1901 [==============================] - 110s 58ms/step - loss: 0.7663 - accuracy: 0.0229
Epoch 4/100
1901/1901 [==============================] - 110s 58ms/step - loss: 0.7515 - accuracy: 0.0244
Epoch 5/100
1901/1901 [==============================] - 107s 56ms/step - loss: 0.7410 - accuracy: 0.0259
Epoch 6/100
1901/1901 [==============================] - 110s 58ms/step - loss: 0.7280 - accuracy: 0.0277
Epoch 7/100
1901/1901 [==============================] - 109s 58ms/step - loss: 0.7355 - accuracy: 0.0296
Epoch 8/100
1901/1901 [==============================] - 112s 59ms/step - loss: 0.7136 - accuracy: 0.0311
Epoch 9/100
1901/1901 [==============================] - 111s 59ms/step - loss: 0.7097 - accuracy: 0.0322
Epoch 10/100
1901/1901 [======================

# わかったこと

- Scalerの導入で学習するようになった
- デフォルトパラメタのままでは20epoch程度で学習が止まってしまう
- fitを変数定義しないと学習曲線を残せない(Kerasの仕様？)

# 次にやること
- sklearnのラッパーでくるんでグリッドサーチを行う epochは20でいい
- aeに渡す引数をちゃんとそろえる